# Stock with TA Indicators

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
import os
import platform 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import sqlite3
import datetime

In [3]:
import yfinance as yf
import option
import utils

In [4]:
# import your own library
from stock import Stock
# from stock_AP import Stock

In [5]:
opt = option.Option()
# set default settings

# we use relative path here, you can set it with a full path
opt.data_dir = "./data"
opt.output_dir = os.path.join(opt.data_dir, "daily")
opt.sqlite_db = os.path.join(opt.data_dir, "sqlite/Equity.db")

# override default option if needed, otherwise use the default settings

In [6]:
print(os.getcwd())

/Users/fuadhassan/Desktop/Labs


In [7]:
# check if all the attributes of the option object
print(vars(opt))

{'name': 'standard', 'user': 'fuadhassan', 'cli_args': {}, 'environ': 'dev', 'verbose': False, 'force': False, 'start_date': None, 'end_date': None, 'tickers': None, 'port_name': None, 'data_dir': './data', 'output_dir': './data/daily', 'sqlite_db': './data/sqlite/Equity.db'}


In [8]:
ticker = 'AAPL'
db_connection = sqlite3.connect(opt.sqlite_db)
stock = Stock(opt, db_connection, ticker)

In [9]:
start_date = "2020-10-1"
end_date = "2024-10-1"

df = stock.get_daily_hist_price_from_db(start_date, end_date)
df.head()

,Ticker,AsOfDate,Open,High,Low,Close,Volume,Dividend,StockSplits
Date,,,,,,,,,
2020-10-01,AAPL,2020-10-01,114.963088,115.041269,113.194277,114.132431,116120400.0,0.0,0
2020-10-02,AAPL,2020-10-02,110.321165,112.744735,109.666413,110.448204,144712000.0,0.0,0
2020-10-05,AAPL,2020-10-05,111.317961,113.995610,110.966153,113.849022,106243800.0,0.0,0
2020-10-06,AAPL,2020-10-06,113.067233,113.477681,109.695741,110.585037,161498200.0,0.0,0
2020-10-07,AAPL,2020-10-07,112.011803,112.920640,111.532947,112.461334,96849000.0,0.0,0


In [10]:
df.tail()

,Ticker,AsOfDate,Open,High,Low,Close,Volume,Dividend,StockSplits
Date,,,,,,,,,
2024-09-25,AAPL,2024-09-25,224.929993,227.289993,224.020004,226.369995,42308700.0,0.0,0
2024-09-26,AAPL,2024-09-26,227.300003,228.500000,225.410004,227.520004,36636700.0,0.0,0
2024-09-27,AAPL,2024-09-27,228.460007,229.520004,227.300003,227.789993,34026000.0,0.0,0
2024-09-30,AAPL,2024-09-30,230.039993,233.000000,229.649994,233.000000,54541900.0,0.0,0
2024-10-01,AAPL,2024-10-01,229.520004,229.649994,223.740005,226.210007,63285000.0,0.0,0


***Let's do some data visualization***

In [11]:
import plotly.graph_objects as go

In [12]:
fig = go.Figure(data=go.Scatter(x=df.index,y=df['Close'], mode='lines'))
fig.show()

***Now plot the daily price as Candle stick***

https://coderzcolumn.com/tutorials/data-science/candlestick-chart-in-python-mplfinance-plotly-bokeh

In [13]:
import plotly.graph_objects as go

candlestick = go.Candlestick(
                            x=df.AsOfDate,
                            open=df['Open'],
                            high=df['High'],
                            low=df['Low'],
                            close=df['Close'],
                            name = ticker
                            )

traces = []
traces.append(candlestick)

layout = {"title": "{} Price".format(ticker)}
fig = go.Figure(data=traces, layout=layout)

fig.show()


### Now call the method to calculate all the Technical Indicators###

In [14]:
stock.calc_all_TA_indicators()
df = stock.ohlcv_df
print(df.tail(10))

           Ticker    AsOfDate        Open        High         Low       Close  \
Date                                                                            
2024-09-18   AAPL  2024-09-18  217.550003  222.710007  217.539993  220.690002   
2024-09-19   AAPL  2024-09-19  224.990005  229.820007  224.630005  228.869995   
2024-09-20   AAPL  2024-09-20  229.970001  233.089996  227.619995  228.199997   
2024-09-23   AAPL  2024-09-23  227.339996  229.449997  225.809998  226.470001   
2024-09-24   AAPL  2024-09-24  228.649994  229.350006  225.729996  227.369995   
2024-09-25   AAPL  2024-09-25  224.929993  227.289993  224.020004  226.369995   
2024-09-26   AAPL  2024-09-26  227.300003  228.500000  225.410004  227.520004   
2024-09-27   AAPL  2024-09-27  228.460007  229.520004  227.300003  227.789993   
2024-09-30   AAPL  2024-09-30  230.039993  233.000000  229.649994  233.000000   
2024-10-01   AAPL  2024-10-01  229.520004  229.649994  223.740005  226.210007   

                 Volume  Di

In [15]:
print(df.columns)

Index(['Ticker', 'AsOfDate', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Dividend', 'StockSplits', 'SMA_10', 'SMA_20', 'SMA_50', 'SMA_200',
       'RSI', 'MACDline', 'MACDsignal', 'BBupperBand', 'BBlowerBand',
       'BBmiddleBand', 'ADX', 'DMP', 'DMN'],
      dtype='object')


### Question 1: Plot the 10, 20, 50, 200 Simple Moving Averages along with the CandleSticks

In [16]:
# Type your code
candlestick = go.Candlestick(
                            x=df.AsOfDate,
                            open=df['Open'],
                            high=df['High'],
                            low=df['Low'],
                            close=df['Close'],
                            name = ticker
                            )
traces = []
traces.append(candlestick)
smas = [10,20,50,200]
for sma in smas:
    traces.append(go.Scatter(
        x=df.AsOfDate,
        y=df[f'SMA_{sma}'],
        mode='lines',
        name=f'SMA {sma}'
    ))

layout = {"title": "{} Price with SMAs".format(ticker),
            "xaxis_title":"Date",
            "yaxis_title":"Price",
          }
fig = go.Figure(data=traces, layout=layout)

fig.show()

### Question 2: Plot RSI with additional 70 and 30 horizontal lines

In [17]:
# Type your code here
RSI = go.Scatter(
        x=df.AsOfDate,
        y=df['RSI'],
        mode='lines',
        name=f'RSI'
)
line_30 = go.Scatter(
    x=df.AsOfDate,
    y=[30]*len(df),
    mode='lines',
    name=f'OverSOLD (30)',
    line=dict( color='red')
)

line_70 = go.Scatter(
    x=df.AsOfDate,
    y=[70]*len(df),
    mode='lines',
    name=f'OverBOUGHT (70)',
    line=dict( color='red')
)

traces = [RSI,line_30,line_70]

layout = go.Layout(
    title=f"{ticker} Price with RSI",
    xaxis_title="Date",
    yaxis_title="RSI",
    # yaxis=dict(range=[0, 100])
    )


fig = go.Figure(data=traces, layout=layout)
fig.show()

In [18]:
print(df.columns)

Index(['Ticker', 'AsOfDate', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Dividend', 'StockSplits', 'SMA_10', 'SMA_20', 'SMA_50', 'SMA_200',
       'RSI', 'MACDline', 'MACDsignal', 'BBupperBand', 'BBlowerBand',
       'BBmiddleBand', 'ADX', 'DMP', 'DMN'],
      dtype='object')


### Question 3: Plot the MACD along with its signal line

In [19]:
# Type your code here
MACDline = go.Scatter(
    x=df.AsOfDate,
    y=df['MACDline'],
    mode='lines',
    name=f'MACDline',
    line=dict( color='red')
)

MACDsignal = go.Scatter(
    x=df.AsOfDate,
    y=df['MACDsignal'],
    mode='lines',
    name=f'MACDsignal',
    line=dict(color='blue')
)
hist = go.Bar(
    x=df['AsOfDate'],
    y=df['MACDline'] - df['MACDsignal'],
    name='MACD Histogram',
    marker=dict(color='blue')
)


traces = [MACDline,MACDsignal,hist]

layout = go.Layout(
    title=f"{ticker} Price with MACD",
    xaxis_title="Date",
    yaxis_title="MACD",
    )


fig = go.Figure(data=traces, layout=layout)

fig.show()


### Question 4: The MACD line above is from calling the pandas_ta library. Can you calculate MACD without calling padnas_ta, but instead calculate it by using pandas sma function?. Show that the two calculation produces the same result

In [20]:
# Type your code here
fast= 12
slow= 26
signal= 9

df['SMA_12'] = df['Close'].rolling(window=fast).mean()
df['SMA_26'] = df['Close'].rolling(window=slow).mean()

df['MACD_w/o_ta'] = df['SMA_12'] - df['SMA_26']

df['MACD_signal_w/o_ta'] = df['MACD_w/o_ta'].rolling(window=signal).mean()


# Type your code here
MACDline = go.Scatter(
    x=df.AsOfDate,
    y=df['MACD_w/o_ta'],
    mode='lines',
    name=f'MACD_w/o_ta',
    line=dict( color='red')
)

MACDsignal = go.Scatter(
    x=df.AsOfDate,
    y=df['MACD_signal_w/o_ta'],
    mode='lines',
    name=f'MACD_signal_w/o_ta',
    line=dict(color='blue')
)
hist = go.Bar(
    x=df['AsOfDate'],
    y=df['MACD_w/o_ta'] - df['MACD_signal_w/o_ta'],
    name='MACD Histogram',
    marker=dict(color='blue')
)


traces = [MACDline,MACDsignal,hist]

layout = go.Layout(
    title=f"{ticker} Price with MACD w/o ta lib using SMA",
    xaxis_title="Date",
    yaxis_title="MACD",
    )


fig = go.Figure(data=traces, layout=layout)

# Show the RSI plot
fig.show()
